# Analyse des données vélo à Paris
Ce notebook télécharge automatiquement les données depuis le site opendata.paris.fr si elles ne sont pas disponibles localement,
les charge, et effectue différentes analyses.

In [10]:
import pandas as pd
import os

In [11]:
# Chemin pour stocker les données localement
DATA_DIR = "data"
CSV_FILE = f"{DATA_DIR}/velo_paris.csv"
PICKLE_FILE = f"{DATA_DIR}/velo_paris.pkl"

# URL des données (OpenData Paris)
DATA_URL = "https://opendata.paris.fr/explore/dataset/comptage-velo-donnees-compteurs/download/?format=csv&timezone=Europe/Paris&use_labels_for_header=true"

In [12]:
os.makedirs(DATA_DIR, exist_ok=True)

In [15]:
if os.path.exists(PICKLE_FILE):
    print("Chargement du DataFrame depuis le pickle...")
    df = pd.read_pickle(PICKLE_FILE)
elif os.path.exists(CSV_FILE):
    print("Chargement du CSV local...")
    df = pd.read_csv(CSV_FILE, sep=',')
    df.to_pickle(PICKLE_FILE)
else:
    print("Téléchargement du CSV depuis OpenData Paris...")
    df = pd.read_csv(DATA_URL, sep=',')
    df.to_csv(CSV_FILE, index=False)
    df.to_pickle(PICKLE_FILE)

df.head()
df[['Comptage horaire']].describe()

Chargement du DataFrame depuis le pickle...


,Comptage horaire
count,972456.000000
mean,81.555008
std,109.786740
min,0.000000
25%,12.000000
50%,45.000000
75%,103.000000
max,3070.000000
